In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [12]:
seed = 1
batch_size = 64
test_batch_size = 1000
momentum = 0.9
epochs = 20
log_interval = 100
lr = 0.01
save_model = False
num_classes = 10
criterion = nn.CrossEntropyLoss()
torch.backends.cudnn.benchmark=True

In [13]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.fc_size = 8 * 8 * 256
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.2),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(self.fc_size, 256),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.2),
            nn.Linear(256, num_classes),
        )
        

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [14]:
def train(model, device, train_loader, optimizer, log_interval, epoch, epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, epochs, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [15]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
use_cuda =  torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)


train_dataset=datasets.CIFAR10('./data_c', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
                       
                   ]))

test_dataset=datasets.CIFAR10('./data_c', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
    ]))

image, label = train_dataset[0]
print(image.size())
print(label)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [17]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=test_batch_size, shuffle=False, num_workers=2)
for images, labels in train_loader:
    print(images.size())
    print(images[0].size())
    print(labels.size())
    
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])


In [18]:
model = CNNNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=5e-4)

def run():
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, log_interval, epoch, epochs)
        test(model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cnn.pt")

In [19]:
%%time

run()

Train Epoch: 1/20 [0/50000 (0%)]	Loss: 2.302331
Train Epoch: 1/20 [6400/50000 (13%)]	Loss: 2.110743
Train Epoch: 1/20 [12800/50000 (26%)]	Loss: 1.963974
Train Epoch: 1/20 [19200/50000 (38%)]	Loss: 1.656430
Train Epoch: 1/20 [25600/50000 (51%)]	Loss: 1.734644
Train Epoch: 1/20 [32000/50000 (64%)]	Loss: 1.521465
Train Epoch: 1/20 [38400/50000 (77%)]	Loss: 1.405634
Train Epoch: 1/20 [44800/50000 (90%)]	Loss: 1.441278

Test set: Average loss: 0.0014, Accuracy: 4859/10000 (49%)

Train Epoch: 2/20 [0/50000 (0%)]	Loss: 1.705333
Train Epoch: 2/20 [6400/50000 (13%)]	Loss: 1.328960
Train Epoch: 2/20 [12800/50000 (26%)]	Loss: 1.344746
Train Epoch: 2/20 [19200/50000 (38%)]	Loss: 1.292666
Train Epoch: 2/20 [25600/50000 (51%)]	Loss: 1.300901
Train Epoch: 2/20 [32000/50000 (64%)]	Loss: 1.284314
Train Epoch: 2/20 [38400/50000 (77%)]	Loss: 1.277954
Train Epoch: 2/20 [44800/50000 (90%)]	Loss: 1.359057

Test set: Average loss: 0.0011, Accuracy: 6096/10000 (61%)

Train Epoch: 3/20 [0/50000 (0%)]	Loss: 1.0

Train Epoch: 18/20 [0/50000 (0%)]	Loss: 0.232355
Train Epoch: 18/20 [6400/50000 (13%)]	Loss: 0.182872
Train Epoch: 18/20 [12800/50000 (26%)]	Loss: 0.252361
Train Epoch: 18/20 [19200/50000 (38%)]	Loss: 0.182098
Train Epoch: 18/20 [25600/50000 (51%)]	Loss: 0.118063
Train Epoch: 18/20 [32000/50000 (64%)]	Loss: 0.087268
Train Epoch: 18/20 [38400/50000 (77%)]	Loss: 0.135847
Train Epoch: 18/20 [44800/50000 (90%)]	Loss: 0.168956

Test set: Average loss: 0.0009, Accuracy: 7408/10000 (74%)

Train Epoch: 19/20 [0/50000 (0%)]	Loss: 0.149065
Train Epoch: 19/20 [6400/50000 (13%)]	Loss: 0.122715
Train Epoch: 19/20 [12800/50000 (26%)]	Loss: 0.159983
Train Epoch: 19/20 [19200/50000 (38%)]	Loss: 0.164055
Train Epoch: 19/20 [25600/50000 (51%)]	Loss: 0.299065
Train Epoch: 19/20 [32000/50000 (64%)]	Loss: 0.166660
Train Epoch: 19/20 [38400/50000 (77%)]	Loss: 0.181905
Train Epoch: 19/20 [44800/50000 (90%)]	Loss: 0.149366

Test set: Average loss: 0.0010, Accuracy: 7364/10000 (74%)

Train Epoch: 20/20 [0/5000